In [1]:
%load_ext autoreload
%autoreload 2
from tf_activation import DeepGraph
from tf_activation.models import mnist_cff as mnist_model
from tf_activation.experiments import distance_finder
from tf_activation.models.mnist_map import mnist_map

import math
import random
import os
from os import listdir
from os.path import isfile

import networkx as nx
import tensorflow as tf
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt

persistence_module = tf.load_op_library('/home/tgebhart/python/tensorflow/bazel-bin/tensorflow/core/user_ops/nn_graph_persistence.so')

from tensorflow.examples.tutorials.mnist import input_data

In [2]:
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.5
config.gpu_options.allocator_type = 'BFC'
config.log_device_placement = True

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [3]:
model = 'mnist_cff_2000.ckpt'
SAVE_PATH = '../logdir/models'
ELITE_LOC = '../logdir/elites/mnist_test_elites_19:45:10_18-09-17'
DIAG_DIR = os.path.join(ELITE_LOC, 'diagrams')
if not os.path.exists(DIAG_DIR):
    os.makedirs(DIAG_DIR)
ED_DIRECTORY = os.path.join(DIAG_DIR, 'elites')
if not os.path.exists(ED_DIRECTORY):
    os.makedirs(ED_DIRECTORY)
TD_DIRECTORY = os.path.join(DIAG_DIR, 'true')
if not os.path.exists(TD_DIRECTORY):
    os.makedirs(TD_DIRECTORY)
csvs = [f for f in listdir(ELITE_LOC) if isfile(os.path.join(ELITE_LOC, f))]
p = 99

In [ ]:
def break_filename(f):
    cls = int(f[0])
    pidx = f.index('.')
    uidx = f.index('_')
    idx = int(f[uidx+1:pidx])
    fname = f[:pidx]
    return cls, idx, fname

In [ ]:
with tf.device('/cpu:0'):
        # Create the model
        x = tf.placeholder(tf.float32, [None, 784])

        # Define loss and optimizer
        y_ = tf.placeholder(tf.float32, [None, 10])

        # Build the graph for the deep net
        net, keep_prob = mnist_model.deepnn(x)

        cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=net['y_conv']))
        train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
        correct_prediction = tf.equal(tf.argmax(net['y_conv'], 1), tf.argmax(y_, 1))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

        saver = tf.train.Saver()

df = []
        
with tf.Session(config=config) as sess:

    saver.restore(sess, os.path.join(SAVE_PATH, model))

    for i in range(len(csvs)):
        cls, idx, fname = break_filename(csvs[i])
        c = np.genfromtxt(os.path.join(ELITE_LOC, fname + '.csv'), delimiter=',')
        col = {}
        col['correct'] = np.argmax(mnist.train.labels[idx])
        
        dfile = os.path.join(ED_DIRECTORY, fname + '.csv')
        dfile_o = os.path.join(TD_DIRECTORY, fname + '.csv')
        
        print(dfile)

        test_inputs = np.stack((mnist.train.images[idx], c))
        test_labels = np.stack((mnist.train.labels[idx], mnist.train.labels[idx]))

        percentiles = persistence_module.layerwise_percentile([net['input'],
                                                            net['W_conv1'],
                                                            net['h_conv1'],
                                                            net['h_conv1'],
                                                            net['W_fc1'],
                                                            net['h_fc1'],
                                                            net['h_fc1_drop'],
                                                            net['W_fc2'],
                                                            net['y_conv']],
                                                            [0, 1, 2, 2, 1, 4, 4, 1, 4],
                                                            [p,p,p])

        ps1 = percentiles.eval(feed_dict={x: test_inputs[0:1], keep_prob:1.0})
        ps2 = percentiles.eval(feed_dict={x: test_inputs[1:2], keep_prob:1.0})
        
        resultF = persistence_module.input_graph_persistence([net['input'],
                                                            net['W_conv1'],
                                                            net['h_conv1'],
                                                            net['h_conv1'],
                                                            net['W_fc1'],
                                                            net['h_fc1'],
                                                            net['h_fc1_drop'],
                                                            net['W_fc2'],
                                                            net['y_conv']],
                                                            [0, 1, 2, 2, 1, 4, 4, 1, 4],
                                                            np.stack((ps1,ps2)), dfile)

        ceF = cross_entropy.eval(feed_dict={x:test_inputs[1:], y_:test_labels[1:], keep_prob:1.0})
        y_convF = sess.run(net['y_conv'], feed_dict={x:test_inputs[1:], keep_prob:1.0})
        accF = accuracy.eval(feed_dict={x:test_inputs[1:], y_:test_labels[1:], keep_prob:1})
        y_convF = y_convF / np.linalg.norm(y_convF)
        
        col['diagramF'] = resultF.eval(feed_dict={x:test_inputs[1:], y_:test_labels[1:], keep_prob:1.0})
        col['cross_entropyF'] = ceF
        col['y_convF'] = y_convF[0,np.argmax(test_labels[1], axis=0)]
        col['accuracyF'] = accF
                
        resultT = persistence_module.input_graph_persistence([net['input'],
                                                            net['W_conv1'],
                                                            net['h_conv1'],
                                                            net['h_conv1'],
                                                            net['W_fc1'],
                                                            net['h_fc1'],
                                                            net['h_fc1_drop'],
                                                            net['W_fc2'],
                                                            net['y_conv']],
                                                            [0, 1, 2, 2, 1, 4, 4, 1, 4],
                                                            np.stack((ps1,ps2)), dfile_o)

        ceT = cross_entropy.eval(feed_dict={x:test_inputs[0:1], y_:test_labels[0:1], keep_prob:1.0})
        y_convT = sess.run(net['y_conv'], feed_dict={x:test_inputs[0:1], keep_prob:1.0})
        accT = accuracy.eval(feed_dict={x:test_inputs[0:1], y_:test_labels[0:1], keep_prob:1})
        y_convT = y_convT / np.linalg.norm(y_convT)
        
        col['diagramT'] = resultT.eval(feed_dict={x:test_inputs[0:1], y_:test_labels[0:1], keep_prob:1.0})
        col['cross_entropyT'] = ceT
        col['y_convT'] = y_convT[0,np.argmax(test_labels[0], axis=0)]
        col['accuracyT'] = accT
                
        df.append(col)

df = pd.DataFrame(df)
df.to_pickle(DIAG_DIR)

INFO:tensorflow:Restoring parameters from ../logdir/models/mnist_cff_2000.ckpt
../logdir/elites/mnist_test_elites_19:45:10_18-09-17/diagrams/elites/6_1491.csv
[[ 0.7217958   0.57106268]
 [ 0.71074486  0.57714272]
 [ 0.68490291  0.53761417]
 [ 0.65999931  0.53662866]
 [ 0.61877108  0.43899426]
 [ 0.60289395  0.39605686]
 [ 0.55824292  0.5369609 ]
 [ 0.20522633  0.17267613]
 [ 0.20017539  0.17378819]
 [ 0.19332062  0.18792623]
 [ 0.19176102  0.        ]
 [ 0.19142087  0.17162202]
 [ 0.19100605  0.05147848]
 [ 0.19100605  0.05147848]
 [ 0.19100605  0.05147848]
 [ 0.19100605  0.05147848]
 [ 0.19100605  0.05147848]
 [ 0.19100605  0.05147848]
 [ 0.19100605  0.05147848]
 [ 0.19100605  0.05147848]
 [ 0.19100605  0.05147848]
 [ 0.19100605  0.05147848]
 [ 0.19100605  0.05147848]
 [ 0.19100605  0.04050198]
 [ 0.19100605  0.05147848]
 [ 0.19100605  0.05147848]
 [ 0.19100605  0.05147848]
 [ 0.19100605  0.04822292]
 [ 0.19100605  0.04537431]
 [ 0.19100605  0.05147848]
 [ 0.19100605  0.05147848]
 [ 0

[[  7.43364096e-01   1.52344033e-01]
 [  6.91825688e-01   1.64139390e-01]
 [  6.38444304e-01   1.39577240e-01]
 [  5.58176517e-01   5.00768244e-01]
 [  5.26051283e-01   4.95798349e-01]
 [  4.92566824e-01   1.67794332e-01]
 [  2.09871024e-01   1.73795223e-01]
 [  1.92515969e-01   0.00000000e+00]
 [  1.91761017e-01   2.23819464e-02]
 [  1.91761017e-01   5.16819507e-02]
 [  1.91761017e-01   0.00000000e+00]
 [  1.91761017e-01   0.00000000e+00]
 [  1.91761017e-01   4.13048677e-02]
 [  1.91761017e-01   5.16819507e-02]
 [  1.91761017e-01   0.00000000e+00]
 [  1.91761017e-01   3.66250030e-03]
 [  1.91761017e-01   4.06944484e-04]
 [  1.91761017e-01   0.00000000e+00]
 [  1.91761017e-01   1.44465286e-02]
 [  1.91761017e-01   0.00000000e+00]
 [  1.91761017e-01   5.16819507e-02]
 [  1.91761017e-01   4.00840342e-02]
 [  1.91761017e-01   0.00000000e+00]
 [  1.91761017e-01   5.16819507e-02]
 [  1.91761017e-01   5.16819507e-02]
 [  1.91761017e-01   0.00000000e+00]
 [  1.91761017e-01   0.00000000e+00]
 

In [ ]:
df = pd.DataFrame(df)

In [ ]:
df.loc[0]['diagramF'].shape